In [1]:
import os
key = os.urandom(16)
for i in range (0,len(key)):
    print(key[i])

159
34
39
25
129
234
132
153
77
12
114
49
194
18
139
204


In [2]:
sub_Box = [#OriginalNib
          [0b0000, 0b0001, 0b0010, 0b0011, 0b0100, 0b0101, 0b0110, 0b0111], 
          #Subistituation
          [0b1001, 0b0100, 0b1010, 0b1011, 0b1101, 0b0001, 0b1000, 0b0101],
          #OriginalNib
          [0b1000, 0b1001, 0b1010, 0b1011, 0b1100, 0b1101, 0b1110, 0b1111], 
          #Subistituation
          [0b0110, 0b0010, 0b0000, 0b0011, 0b1100, 0b1110, 0b1111, 0b0111]]

In [3]:
for i in range (0, len(sub_Box)):
        for y in range(0, len(sub_Box[i])):
            sub_Box[i][y] = format(sub_Box[i][y], "04b")

In [4]:
#word4 for = word0 xor subNib(rotNib(word3))
def rotNib(key):
    temp1 = key[0]
    temp2 = key[1]
    
    key[0] = key[2]
    key[1] = key[3]
    
    key[2] = temp1
    key[3] = temp2
    
    return key

In [5]:
def SubNib(key):    
    for i in range(0, len(key)):
        Div1 = key[i][:4]
        Div2 = key[i][4:]
        
        flagDiv1 = 0
        flagDiv2 = 0
        
        y = 0
        while y < len(sub_Box):
            for z in range(0, len(sub_Box[y])):
                if flagDiv1 == 0:
                    if key[i][:4] == sub_Box[y][z]:
                        Div1 = sub_Box[y + 1][z]
                        flagDiv1 = 1
                if flagDiv2 == 0:
                    if key[i][4:] == sub_Box[y][z]:
                        Div2 = sub_Box[y + 1][z]
                        flagDiv2 = 1
                
                if flagDiv1 == 1 & flagDiv2 == 1:
                    key[i] = Div1 + Div2
                    break
            y = y + 2
    return key

In [6]:
def decimalToBinary(n):
    # converting decimal to binary
    # and removing the prefix(0b)
    return '{:032b}'.format(n)

In [7]:
def ExpandKey(key, sub_Box):
    KeyArray = []
    for i in range (0,len(key)):
        KeyArray.append(format(key[i], "08b"))
    
    #for i in range (0,len(KeyArray)):
     #   print(KeyArray[i])
        
        
    MultiArrayKey = []
    rows,cols = (4,4)
    i = 0;
    for r in range(rows):
        col = []
        for c in range(cols):
            col.append(KeyArray[i])
            i+=1
        MultiArrayKey.append(col)

    #print(MultiArrayKey)
    FW = 0
    
    for i in range (0, 9):
        
        print('last word: ', MultiArrayKey[len(MultiArrayKey ) - 1])
        Rotated_Word = rotNib(MultiArrayKey[len(MultiArrayKey ) - 1])
        print('rotated word: ', Rotated_Word)
        
        subistutedWord = SubNib(Rotated_Word)
        
        print('subistuted Word', subistutedWord)
        
        FirstWordInt = int(MultiArrayKey[FW][0] + MultiArrayKey[FW][1] +  MultiArrayKey[FW][2] + MultiArrayKey[FW][3], 2)
        SecondWordInt = int(subistutedWord[0] + subistutedWord[1] + subistutedWord[2] + subistutedWord[3], 2)
        
        print('First Word Int: ', FirstWordInt)
        print('Second Word Int: ', SecondWordInt)
        
        new_Word = FirstWordInt ^ SecondWordInt
    
        new_Word = decimalToBinary(new_Word)
        
        print(new_Word)
        
        col = []
        
        col.append(new_Word[:8])
        col.append(new_Word[8:16])
        col.append(new_Word[16:24])
        col.append(new_Word[24:])
        
        MultiArrayKey.append(col)
        
        for z in range (0, 3):
            
            FW += 1
            num = len(MultiArrayKey) - 1
            
            FirstWordInt = int(MultiArrayKey[FW][0] + MultiArrayKey[FW][1] +  MultiArrayKey[FW][2] + MultiArrayKey[FW][3], 2)
            SecondWordInt = int(MultiArrayKey[num][0] + MultiArrayKey[num][1] +  MultiArrayKey[num][2] + MultiArrayKey[num][3], 2)
            
            new_Word = FirstWordInt ^ SecondWordInt
            
            new_Word = decimalToBinary(new_Word)
            
            print(new_Word)
            
            col = []
            
            col.append(new_Word[:8])
            col.append(new_Word[8:16])
            col.append(new_Word[16:24])
            col.append(new_Word[24:])
            
            MultiArrayKey.append(col)
        
        
    for i in range (0, len(MultiArrayKey)):
        print('Word number', i ,':', MultiArrayKey[i])
    return MultiArrayKey

In [8]:
def text2Unicode(text):
    w, h = 16, 8
    l = [0 for x in range(16)] 

    for i in range(16):
        
        
        l1 = ' '.join(map(bin,bytearray(text[i],'utf8')))
        l[i] = l1[2:]
        #l = list(l1[2:])
        #for i1 in range(len(l1)):
        #    lst2[i][i1]=l[i1]
        #    print(l[i1])
    w, h = 16, 8
    lst2 = [[0 for x in range(8)] for y in range(16)] 

    for row in range(16):
        r1 = list(l[row])
        for i in range(0, len(r1)):
            lst2[row][i] = int(r1[i]) 

    return lst2

In [9]:
def inv_mix_col(a, b):
    if b == 9:
        return gmul(gmul(gmul(a,2), 2), 2) ^ a
    if b == 11:
        temp = gmul(a, 2)
        temp = gmul(temp, 2)
        temp = temp ^ a
        temp = gmul(temp, 2)
        temp = temp ^ a
        return temp
    if b == 13:
        temp = gmul(a, 2)
        temp = temp ^ a
        temp = gmul(temp, 2)
        temp = gmul(temp, 2)
        temp = temp ^ a
        return temp
    if b == 14:
        temp = gmul(a, 2)
        temp = temp ^ a
        temp = gmul(temp, 2)
        temp = temp ^ a
        temp = gmul(temp, 2)
        return temp
def gmul(a, b):
    if b == 1:
        return a
    binary = '{:08b}'.format(a)
    
    tmp = (a << 1) & 0xff
    print('temp After', binary)
    
    if b == 2:
        if a < 128:
            #binary = '{:08b}'.format(a)
            #print('a < 128:', binary)
            return tmp
        else:
            #binary = '{:0b}'.format(a)
            #print('a > 128:', binary)
            return tmp ^ 0x1b
        
#        return tmp if a < 128 else tmp & 0xff ^ 0x1b
    if b == 3:
        return gmul(a, 2) ^ a

def printHex(val):
    return print('{:02x}'.format(val), end=' ')

def inv_mixColumns(mat1):
    L1 = [0 for x in range(16)]   
    for i1 in range(0,16):
        firstnumber = ""
        for i in range(0,8):
            firstnumber += str(mat1[i1][i])

        L1[i1] =  firstnumber            
    print(L1)
    hexadecimal_string = [0 for x in range(16)]
    for i1 in range(0,16):
        decimal_representation = int(L1[i1], 2)
        hexadecimal_string[i1] = hex(decimal_representation)
        hexadecimal_string[i1] = int(hexadecimal_string[i1],16)

    print(hexadecimal_string)
    ii = 0
    l3 = [0 for x in range(16)]
    for i1 in range(0,4):
        l3[ii] = inv_mix_col(hexadecimal_string[ii], 0x0E) ^ inv_mix_col(hexadecimal_string[ii+4], 0x0B) ^ inv_mix_col(hexadecimal_string[ii+8], 0x0D) ^ inv_mix_col(hexadecimal_string[ii+12], 0x09)
        l3[ii+4] = inv_mix_col(hexadecimal_string[ii], 0x09) ^ inv_mix_col(hexadecimal_string[ii+4], 0x0E) ^ inv_mix_col(hexadecimal_string[ii+8], 0x0B) ^ inv_mix_col(hexadecimal_string[ii+12], 0x0D)
        l3[ii+8] = inv_mix_col(hexadecimal_string[ii], 0x0D) ^ inv_mix_col(hexadecimal_string[ii+4], 0x09) ^ inv_mix_col(hexadecimal_string[ii+8], 0x0E) ^ inv_mix_col(hexadecimal_string[ii+12], 0x0B)
        l3[ii+12] = inv_mix_col(hexadecimal_string[ii], 0x0B) ^ inv_mix_col(hexadecimal_string[ii+4], 0x0D) ^ inv_mix_col(hexadecimal_string[ii+8], 0x09) ^ inv_mix_col(hexadecimal_string[ii+12], 0x0E)
        l3[ii]   ='{:08b}'.format(l3[ii])
        l3[ii+4] ='{:08b}'.format(l3[ii+4])
        l3[ii+8] ='{:08b}'.format(l3[ii+8])
        l3[ii+12]='{:08b}'.format(l3[ii+12])
        ii +=1
    print(l3)
    ii = 0
    w, h = 8, 16
    lst2 = [[0 for x in range(w)] for y in range(h)]
    res = [[0 for x in range(w)] for y in range(h)]
    for row in range(16):
        r1 = list(l3[row])
        for i in range(0, len(r1)):
            lst2[row][i] = int(r1[i]) 
    return lst2

In [10]:
def XOR(Key1,plaintext):
    w= 4
    w, h = 16, 8
    lst2 = [[0 for x in range(8)] for y in range(16)] 
    w, h = 16, 8
    lst3 = [[0 for x in range(8)] for y in range(16)] 
    L2 = [0 for x in range(w)]   
    r1 = [0 for x in range(8)]   
    r2 = [0 for x in range(8)]   
    for i in range(0,16):
        io = 0
        for i1 in range(0,8):
            L3 = " "
            binOne = plaintext[i][i1]
            binTwo = Key1[i][i1]
            binOne1 = str(binOne)
            binTwo1 = str(binTwo)
            L33 = int(binOne1,2) ^ int(binTwo1,2)
            L3 = bin(L33)
            r = str(L3[2:])
            lst3[i][i1] = int(r)
            #    print("\nResult = ",lst3)
    return lst3

In [11]:
def inv_Sub_Nib(Roundkey,inv_sboxIN,inv_sboxOUT):
    w, h = 16, 8
    lst2 = [[0 for x in range(w)] for y in range(h)] 
    i2 = 0
    for i1 in range(0, 16):
        if (Roundkey[i2][0] == inv_sboxIN[i1][0] and Roundkey[i2][1] == inv_sboxIN[i1][1] and Roundkey[i2][2] == inv_sboxIN[i1][2] and Roundkey[i2][3] == inv_sboxIN[i1][3]):
            for i in range(0, 4):
                 Roundkey[i2][i] = inv_sboxOUT[i1][i]
                    
            i2 += 1
    i3 = 0
    for i1 in range(0, 16):
        if (Roundkey[i3][4] == inv_sboxIN[i1][0] and Roundkey[i3][5] == inv_sboxIN[i1][1] and Roundkey[i3][6] == inv_sboxIN[i1][2] and Roundkey[i3][7] == inv_sboxIN[i1][3]):
            i4 = 0
            for i in range(4, 8):
                Roundkey[i3][i] = inv_sboxOUT[i1][i4]
                i4 += 1

            i3 += 1
    return Roundkey

In [12]:
import numpy as np

def inv_Shift_Row(inv_shift):
    w, h = 16, 8
    lst2 = [[0 for x in range(w)] for y in range(h)] 
    for i in range(0, 8):
        lst2[0][i] = inv_shift[4][i]
        inv_shift[4][i] = inv_shift[5][i]
        inv_shift[5][i] = inv_shift[6][i]
        inv_shift[6][i] = inv_shift[7][i]
        inv_shift[7][i] = lst2[0][i]
        ##2nd row
        lst2[1][i] =  inv_shift[8][i]
        inv_shift[8][i] = inv_shift[10][i]
        inv_shift[10][i] = lst2[1][i]
        ##2nd row net column
        lst2[2][i] =  inv_shift[9][i]
        inv_shift[9][i] = inv_shift[11][i]
        inv_shift[11][i] = lst2[2][i]
        ##4th row
        lst2[2][i] = inv_shift[12][i]
        inv_shift[12][i] = inv_shift[15][i]
        lst2[3][i] = inv_shift[13][i]
        inv_shift[13][i] = lst2[2][i]
        lst2[4][i] = inv_shift[14][i]
        inv_shift[14][i] = lst2[3][i]
        lst2[5][i] = inv_shift[15][i]
        inv_shift[15][i] = lst2[4][i]
    return inv_shift

In [13]:
inv_sboxOUT = [[0,0,0,0],#1 
          [0,0,0,1],#2 
          [0,0,1,0],#3 
          [0,0,1,1],#4 
          [0,1,0,0],#5 
          [0,1,0,1],#6 
          [0,1,1,0],#7 
          [0,1,1,1],#8 
          [1,0,0,0],#9 
          [1,0,0,1],#10
          [1,0,1,0],#11
          [1,0,1,1],#12
          [1,1,0,0],#13
          [1,1,0,1],#14
          [1,1,1,0],#15
          [1,1,1,1]]#16

inv_sboxIN = [[1,0,0,1],#1 
            [0,1,0,0],#2 
            [1,0,1,0],#3 
            [1,0,1,1],#4 
            [1,1,0,1],#5 
            [0,0,0,1],#6 
            [1,0,0,0],#7 
            [0,1,0,1],#8 
            [0,1,1,0],#9 
            [0,0,1,0],#10
            [0,0,0,0],#11
            [0,0,1,1],#12
            [1,1,0,0],#13
            [1,1,1,0],#14
            [1,1,1,1],#15
            [0,1,1,1]]#16
MultiArrayKey = ExpandKey(key, sub_Box)

plain_text = input("Enter a string to be encoded : ")
length = len(plain_text)
plain_text_split = []
for i in range(length//16):
    plain_text_split.append(plain_text[0+16*i:16+16*i])
if not length%16==0:        
    plain_text_split.append(plain_text[16*(length//16):])
if len(plain_text_split[-1])<16:
    while(len(plain_text_split[-1])<16):
        plain_text_split[-1]+=' '
for sub_string in plain_text_split : 
    A0 = text2Unicode(sub_string)
    print(A0)

last word:  ['11000010', '00010010', '10001011', '11001100']
rotated word:  ['10001011', '11001100', '11000010', '00010010']
subistuted Word ['01100011', '11001100', '11001010', '01001010']
First Word Int:  2669815577
Second Word Int:  1674365514
11111100111011101110110101010011
01111101000001000110100111001010
00110000000010000001101111111011
01010011110001001101000110110001
last word:  ['01010011', '11000100', '11010001', '10110001']
rotated word:  ['11010001', '10110001', '01010011', '11000100']
subistuted Word ['11100100', '00110100', '00011011', '11001101']
First Word Int:  1674365514
Second Word Int:  3828620237
10000111111110001101000110000111
01111011000101100011110011010100
00000110000100100101010100011110
00110110000110100100111011100101
last word:  ['00110110', '00011010', '01001110', '11100101']
rotated word:  ['01001110', '11100101', '00110110', '00011010']
subistuted Word ['11011111', '11110001', '10111000', '01000000']
First Word Int:  805837819
Second Word Int:  3757160

In [14]:
def encrypt(A0,MultiArrayKey):
    w, h = 16, 8
    lst2 = [[0 for x in range(8)] for y in range(16)] 
    Key2 = [0 for y in range(16)] 
    row1 = 0
    row = 39
    for row2 in range (9):
        Key2[row1] = MultiArrayKey[row][0]
        Key2[row1+1] = MultiArrayKey[row][1]
        Key2[row1+2] = MultiArrayKey[row][2]
        Key2[row1+3] = MultiArrayKey[row][3]
        Key2[row1+4] = MultiArrayKey[row-1][0]
        Key2[row1+5] = MultiArrayKey[row-1][1]
        Key2[row1+6] = MultiArrayKey[row-1][2]
        Key2[row1+7] = MultiArrayKey[row-1][3]
        Key2[row1+8] = MultiArrayKey[row - 2][0]
        Key2[row1+9] = MultiArrayKey[row - 2][1]
        Key2[row1+10] = MultiArrayKey[row - 2][2]
        Key2[row1+11] = MultiArrayKey[row - 2][3]
        Key2[row1+12] = MultiArrayKey[row - 3][0]
        Key2[row1+13] = MultiArrayKey[row - 3][1]
        Key2[row1+14] = MultiArrayKey[row - 3][2]
        Key2[row1+15] = MultiArrayKey[row - 3][3]

        for row in range(16):
                r1 = list(Key2[row])
                for i in range(0, len(r1)):
                    lst2[row][i] = int(r1[i]) 
        print(lst2)
        print(A0)

        plaintext = A0
        Key1 = lst2
        #sub-bytes
        inv_shift = inv_Sub_Nib(plaintext,inv_sboxIN,inv_sboxOUT)
        print("inv_Sub_Nib",inv_shift)
        #inv_shift-rows
        mix=inv_Shift_Row(inv_shift)
        print("inv_Shift_Row",mix)
        #mix-columns
        v1 = inv_mixColumns(mix)
        print("mix_column",v1)
        #round key
        Roundkey = XOR(Key1,v1)
        print("Roundkey",Roundkey)
        plaintext = Roundkey
        text = ""
        for i1 in range(16):
            for i in range(8):
                text+=chr(int(plaintext[i1][i])) # chr converts unicode integer to unicode character
        row -= 4

    return Roundkey

In [15]:
cipher_text = encrypt(A0,MultiArrayKey)
print(cipher_text)

[[1, 0, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 1, 1, 1, 0, 0], [0, 1, 0, 1, 1, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 1, 0, 1, 0, 0, 1], [1, 0, 0, 1, 1, 0, 0, 1], [1, 0, 1, 1, 1, 1, 0, 1], [1, 0, 0, 1, 0, 0, 0, 0], [1, 0, 1, 1, 0, 1, 0, 1], [0, 1, 0, 0, 0, 1, 0, 1], [1, 0, 0, 0, 0, 1, 0, 1], [1, 0, 1, 0, 0, 1, 0, 0], [0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 1, 1, 0, 0, 0]]
[[1, 1, 0, 0, 0, 0, 1, 0], [1, 1, 1, 1, 0, 0, 1, 0], [1, 1, 0, 1, 1, 1, 1, 0], [1, 1, 1, 0, 1, 0, 1, 0], [1, 1, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0]]
inv_Sub_Nib [[1, 1, 0, 0, 1, 0, 0, 1], [1, 1, 1, 0, 0, 0, 1, 0], [1, 1, 0, 1, 1, 1, 1, 0], [1, 1, 1, 0, 1, 0, 1, 0], [1, 1, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0

In [26]:
def BinaryToDecimal(binary):
    string = int(binary, 2)     
    return string
# initializing a empty string for
# storing the string data
str_data =' '
temp_data =' '
# slicing the input and converting it
# in decimal and then converting it in string
for i in range(0, 16):
    temp_data =' ' 
    # slicing the bin_data from index range [0, 6]
    # and storing it in temp_data
#    for i1 in range(0,16):
    for i1 in range(0,8):
        temp_data += str(cipher_text[i][i1])
    # passing temp_data in BinarytoDecimal() function
    # to get decimal value of corresponding temp_data
    decimal_data = BinaryToDecimal(temp_data)
      
    # Decoding the decimal value returned by
    # BinarytoDecimal() function, using chr()
    # function which return the string corresponding
    # character for given ASCII value, and store it
    # in str_data
    str_data = str_data + chr(decimal_data)
 
# printing the result
print("The Binary value after string conversion is:",
       int(str_data[0]))

ValueError: ignored

In [18]:
import binascii
  
# Initializing a ASCII string
Text = str_data
  
# Calling the a2b_uu() function to
# Convert the ascii string to binary
Binary = binascii.a2b_uu(Text)
  
# Getting the Binary value
print(Binary)

ValueError: ignored